# Merge historic odds with other processed matches

This notebook retrieves historic odds for the training/testing period, if possible merges with our procesed matches data set, and finally compare our performance with the bookmakers.

## Load data and dependencies

In [48]:
import os
import json
import numpy as np
import pandas as pd
import datetime as dt

from definitions import ODDS_PATH, RAW_PATH, ROOT_DIR, GEN_PATH
from utilities.helper import logger

# Read configuration file
with open(os.path.join(ROOT_DIR, 'config.json')) as f:
    config = json.load(f)

proc_match_filepath = os.path.join(GEN_PATH, config['proc_match_filename_final'])
odds_filepath = os.path.join(GEN_PATH, config['odds_filename'])

# Read all ATP players
atp_players = pd.read_csv(os.path.join(RAW_PATH, 'atp_players.csv'), parse_dates=['birthdate'])

years = config['proc_year']

# Load all matches from odds files
match_odds = []
for year in range(years['from'], years['to'] + 1):
    if year < 2013:
        match_odds.append(pd.read_excel(os.path.join(ODDS_PATH, str(year) + '.xls'), 
                                          parse_dates=['Date']))
    else:
        match_odds.append(pd.read_excel(os.path.join(ODDS_PATH, str(year) + '.xlsx'), 
                                         parse_dates=['Date']))

match_odds = pd.concat(match_odds, sort=False)

processed_matches = pd.read_hdf(proc_match_filepath, key='matches')

## Initial processing and cleaning of match odds

In [22]:
@logger
def start_pipe(df):
    return df.copy()

@logger
def clean_odds(df):
    filter_cols = ['ATP', 'Date', 'Winner', 'Loser', 'B365W', 'B365L', 'EXW', 'EXL', 'PSW', 'PSL']
    df = df.filter(filter_cols)
    df.columns = map(str.lower, df.columns)
    df.fillna(0, inplace=True)
    df['tourney_date'] = dt.date(1970, 1, 1)
    df['avg_ratio'] = float('nan')
    df['max_w'] = float('nan')
    df['max_l'] = float('nan')
    df['broker_max_w'] = ''
    df['broker_max_l'] = ''
    df['winner'] = df['winner'].str.lower().str.strip()
    df['loser'] = df['loser'].str.lower().str.strip()
    
    return df

@logger
def calc_odds(df):
    t_date = dt.date(1970, 1, 1)
    t_num = 0
    
    def wrapper(match):
        nonlocal t_date, t_num
        
        if match['atp'] != t_num:
            t_num = match['atp']
            t_date = match['date']
        
        return calc_match_odds(match, t_date)
    return df.apply(wrapper, axis=1)

def calc_match_odds(match, t_date):
    match['tourney_date'] = t_date
    
    brokers = ['b365', 'ex', 'ps']
    total_ratio = 0
    n = 0
    max_w = 0
    max_l = 0
    broker_max_w = ''
    broker_max_l = ''
    
    for b in brokers:
        odds_w = match[b + 'w']
        odds_l = match[b + 'l']
        
        if isinstance(odds_w, str):
            odds_w = float(odds_w.replace(',', ''))
        
        if isinstance(odds_l, str):
            odds_l = float(odds_l.replace(',', ''))
        
        if odds_w > 0 and odds_l > 0:
            if odds_w > max_w:
                max_w = odds_w
                broker_max_w = b

            if odds_l > max_l:
                max_l = odds_l
                broker_max_l = b

            total_ratio += odds_w / odds_l
            n += 1
    
    if n > 0:
        match['max_w'] = max_w
        match['max_l'] = max_l
        match['broker_max_w'] = broker_max_w
        match['broker_max_l'] = broker_max_l
        match['avg_ratio'] = total_ratio/n
        
    return match 

@logger
def prune_odds(df):
    brokers = ['b365', 'ex', 'ps']
    
    for b in brokers:
        df.drop([b + 'w', b + 'l'], inplace=True, axis=1)
    
    df.dropna(inplace=True)
    
    return df

proc_match_odds = (match_odds
    .pipe(start_pipe)
    .pipe(clean_odds)
    .pipe(calc_odds)
    .pipe(prune_odds))

proc_match_odds

start_pipe took=0:00:00.004983 shape=(10506, 40)
clean_odds took=0:00:00.037894 shape=(10506, 16)
calc_odds took=0:00:03.197988 shape=(10506, 16)
prune_odds took=0:00:00.011218 shape=(10475, 10)


,atp,date,winner,loser,tourney_date,avg_ratio,max_w,max_l,broker_max_w,broker_max_l
0,1,2016-01-04,dimitrov g.,simon g.,2016-01-04,0.772583,1.68,2.31,ex,ps
1,1,2016-01-04,kudla d.,smith j.p.,2016-01-04,0.665692,1.63,2.40,ps,ps
2,1,2016-01-04,kamke t.,mitchell b.,2016-01-04,0.924223,1.90,2.00,ps,b365
3,1,2016-01-04,chung h.,groth s.,2016-01-04,0.980863,1.93,1.96,ps,ps
4,1,2016-01-05,goffin d.,bellucci t.,2016-01-04,0.366112,1.31,3.74,ps,ps
...,...,...,...,...,...,...,...,...,...,...
2605,66,2019-11-15,nadal r.,tsitsipas s.,2019-11-10,0.475008,1.44,3.26,b365,ps
2606,66,2019-11-15,zverev a.,medvedev d.,2019-11-10,1.097765,2.14,1.90,ps,b365
2607,66,2019-11-16,tsitsipas s.,federer r.,2019-11-10,2.755928,3.75,1.33,ps,ps
2608,66,2019-11-16,thiem d.,zverev a.,2019-11-10,0.888095,1.84,2.10,ps,ps


## Processing of ATP players

In [26]:
@logger
def clean_players(df):
    df = df.filter(['player_id', 'firstname', 'lastname'])
    df['lastname'] = df['lastname'].str.lower()
    df['firstname'] = df['firstname'].str.lower()
    df['fullname'] = ''
    return df

@logger
def transform_name(df):
    return df.apply(transform_player_name, axis=1)

def transform_player_name(player):
    fullname = str(player['lastname']) + ' '
    
    for n in str(player['firstname']).split(' '):
        if len(n) > 0:
            fullname += n[0] + '.'
            
    player['fullname'] = fullname.strip()
    
    return player
    
proc_atp_players = (atp_players
    .pipe(start_pipe)
    .pipe(clean_players)
    .pipe(transform_name))

proc_atp_players

start_pipe took=0:00:00.009112 shape=(54405, 6)
clean_players took=0:00:00.064822 shape=(54405, 4)
transform_name took=0:00:10.671050 shape=(54405, 4)


,player_id,firstname,lastname,fullname
0,100001,gardnar,mulloy,mulloy g.
1,100002,pancho,segura,segura p.
2,100003,frank,sedgman,sedgman f.
3,100004,giuseppe,merlo,merlo g.
4,100005,richard pancho,gonzales,gonzales r.p.
...,...,...,...,...
54400,209366,joe,herin,herin j.
54401,209367,geronimo,marcolini,marcolini g.
54402,209368,juan martin,fumeaux buenaventura,fumeaux buenaventura j.m.
54403,209369,vadim,kontseba,kontseba v.


## Merge attempt by fullname

In [60]:
@logger
def merge_names(df, atp):
    df['winner_id'] = df['winner']
    df['loser_id'] = df['loser']
    
    ws = df['winner'].to_numpy()
    ls = df['loser'].to_numpy()
    ps = np.unique(np.append(ws, ls))
    pids = []
    
    # Match by player name
    for p in ps:
        # Match by full name, exact match
        pid = atp.loc[atp['fullname'] == p]
        
        # If no fullname match, get lastname
        if len(pid) == 0:
            lastname = max(p.split(), key=len)
            pid = atp.loc[atp['lastname'] == lastname]
        
        # Extract ATP info
        if len(pid) == 1:      
            pids.append(pid.iloc[0]['player_id'])
        elif len(pid) > 1:
            pid_ids = ','.join([str(v.player_id) for v in pid.itertuples()])
            pids.append(pid_ids)
        else:
            pids.append(0)
                            
    df['winner_id'] = df['winner_id'].replace(ps, pids)
    df['loser_id'] = df['loser_id'].replace(ps, pids)
    
    return df

proc_match_odds = (proc_match_odds
    .pipe(start_pipe)
    .pipe(merge_names, proc_atp_players))

proc_match_odds

start_pipe took=0:00:00.003031 shape=(10475, 12)
merge_names took=0:00:03.857776 shape=(10475, 12)


,atp,date,winner,loser,tourney_date,avg_ratio,max_w,max_l,broker_max_w,broker_max_l,winner_id,loser_id
0,1,2016-01-04,dimitrov g.,simon g.,2016-01-04,0.772583,1.68,2.31,ex,ps,105777,"104468,200637"
1,1,2016-01-04,kudla d.,smith j.p.,2016-01-04,0.665692,1.63,2.40,ps,ps,106045,105441
2,1,2016-01-04,kamke t.,mitchell b.,2016-01-04,0.924223,1.90,2.00,ps,b365,104735,106093
3,1,2016-01-04,chung h.,groth s.,2016-01-04,0.980863,1.93,1.96,ps,ps,"106184,111202",105032
4,1,2016-01-05,goffin d.,bellucci t.,2016-01-04,0.366112,1.31,3.74,ps,ps,105676,105064
...,...,...,...,...,...,...,...,...,...,...,...,...
2605,66,2019-11-15,nadal r.,tsitsipas s.,2019-11-10,0.475008,1.44,3.26,b365,ps,104745,126774
2606,66,2019-11-15,zverev a.,medvedev d.,2019-11-10,1.097765,2.14,1.90,ps,b365,100644,106421
2607,66,2019-11-16,tsitsipas s.,federer r.,2019-11-10,2.755928,3.75,1.33,ps,ps,126774,103819
2608,66,2019-11-16,thiem d.,zverev a.,2019-11-10,0.888095,1.84,2.10,ps,ps,106233,100644


## Further merge attemps by oppenent player id

In [75]:
@logger
def merge_by_opp(df, atp_matches):
    atp_matches['tourney_date'] = pd.to_datetime(atp_matches['tourney_date'], unit='s')
    return df.apply(merge_by_opp_match, axis=1, args=(atp_matches,))

def merge_by_opp_match(match, atp_matches):
    winner_id = match['winner_id']
    loser_id = match['loser_id']
    
    wid_not_num = not isinstance(winner_id, int)
    lid_not_num = not isinstance(loser_id, int)
    
    if wid_not_num & lid_not_num:
        match['winner_id'] = float('nan')
        match['loser_id'] = float('nan')
    elif wid_not_num or lid_not_num:
        if wid_not_num:
            search_id = loser_id
            search_player = 'loser_id'
            get_player = 'winner_id'
            search_player_atp = 'player_2' # == loser_in in processed matches
            get_player_atp = 'player_1'  # == winner_id in processed matches
            alts = winner_id
        else:
            search_id = winner_id
            search_player = 'winner_id'
            get_player = 'loser_id'
            search_player_atp = 'player_1'  # == winner_id in processed matches
            get_player_atp = 'player_2'  # == loser_id in processed matches
            alts = loser_id
        
        mask = (
            (atp_matches['tourney_date'] == match['tourney_date']) & 
            (atp_matches[search_player_atp] == match[search_player]))
        potential = atp_matches.loc[mask]
        
        if len(potential) == 1:
            potential = potential.iloc[0]
            alts = alts.split(',')
            hit = False
            
            for a in alts:
                player_id = int(a)
                
                if player_id == potential[get_player_atp]:
                    match[get_player] = player_id
                    hit = True
                    break
            
            if not hit:
                match[get_player] = float('nan')
            
        else:
            match[get_player] = float('nan')
    
    return match    
    
merged_match_odds = (proc_match_odds
    .pipe(start_pipe)
    .pipe(merge_by_opp, processed_matches))

merged_match_odds

start_pipe took=0:00:00.004803 shape=(10475, 12)
merge_by_opp took=0:00:06.618051 shape=(10475, 12)


,atp,date,winner,loser,tourney_date,avg_ratio,max_w,max_l,broker_max_w,broker_max_l,winner_id,loser_id
0,1,2016-01-04,dimitrov g.,simon g.,2016-01-04,0.772583,1.68,2.31,ex,ps,105777.0,NaN
1,1,2016-01-04,kudla d.,smith j.p.,2016-01-04,0.665692,1.63,2.40,ps,ps,106045.0,105441.0
2,1,2016-01-04,kamke t.,mitchell b.,2016-01-04,0.924223,1.90,2.00,ps,b365,104735.0,106093.0
3,1,2016-01-04,chung h.,groth s.,2016-01-04,0.980863,1.93,1.96,ps,ps,111202.0,105032.0
4,1,2016-01-05,goffin d.,bellucci t.,2016-01-04,0.366112,1.31,3.74,ps,ps,105676.0,105064.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2605,66,2019-11-15,nadal r.,tsitsipas s.,2019-11-10,0.475008,1.44,3.26,b365,ps,104745.0,126774.0
2606,66,2019-11-15,zverev a.,medvedev d.,2019-11-10,1.097765,2.14,1.90,ps,b365,100644.0,106421.0
2607,66,2019-11-16,tsitsipas s.,federer r.,2019-11-10,2.755928,3.75,1.33,ps,ps,126774.0,103819.0
2608,66,2019-11-16,thiem d.,zverev a.,2019-11-10,0.888095,1.84,2.10,ps,ps,106233.0,100644.0


## Display merge results and fetch final frame

In [76]:
@logger
def prune_matches(df):
    mask = ((np.isnan(df['winner_id'])) | (np.isnan(df['loser_id'])))
    missed = len(df.loc[mask])
    
    print(f"Merged {(1 - missed/len(df))}")
    
    df.dropna(inplace=True)
    
    return df

pruned_match_odds = (merged_match_odds
    .pipe(start_pipe)
    .pipe(prune_matches))

pruned_match_odds

start_pipe took=0:00:00.002691 shape=(10475, 12)
Merged 0.8246300715990453
prune_matches took=0:00:00.014277 shape=(8638, 12)


,atp,date,winner,loser,tourney_date,avg_ratio,max_w,max_l,broker_max_w,broker_max_l,winner_id,loser_id
1,1,2016-01-04,kudla d.,smith j.p.,2016-01-04,0.665692,1.63,2.40,ps,ps,106045.0,105441.0
2,1,2016-01-04,kamke t.,mitchell b.,2016-01-04,0.924223,1.90,2.00,ps,b365,104735.0,106093.0
3,1,2016-01-04,chung h.,groth s.,2016-01-04,0.980863,1.93,1.96,ps,ps,111202.0,105032.0
4,1,2016-01-05,goffin d.,bellucci t.,2016-01-04,0.366112,1.31,3.74,ps,ps,105676.0,105064.0
5,1,2016-01-05,troicki v.,johnson s.,2016-01-04,1.292972,2.37,1.72,ps,b365,104678.0,105449.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2605,66,2019-11-15,nadal r.,tsitsipas s.,2019-11-10,0.475008,1.44,3.26,b365,ps,104745.0,126774.0
2606,66,2019-11-15,zverev a.,medvedev d.,2019-11-10,1.097765,2.14,1.90,ps,b365,100644.0,106421.0
2607,66,2019-11-16,tsitsipas s.,federer r.,2019-11-10,2.755928,3.75,1.33,ps,ps,126774.0,103819.0
2608,66,2019-11-16,thiem d.,zverev a.,2019-11-10,0.888095,1.84,2.10,ps,ps,106233.0,100644.0


## If needed: save merged odds to file

In [77]:
pruned_match_odds.to_hdf(odds_filepath, key='odds', mode='w')